In [26]:
import pandas as pd
import numpy as np
import requests
import os
import glob
from bs4 import BeautifulSoup

# get all xpt file names

In [27]:

# xpt_file_list = glob.glob('./xpt_data/*.XPT')
# print(os.path.basename(xpt_file_list[0]).split('.')[0])

# get variable names

In [28]:
def get_variable_description(url):

    # get the html page with the variable codebook for all xpt files
    # use this to rename the columns to descriptive cols
    page = requests.get(url)

    # beautiful soup is good for parsing html
    soup = BeautifulSoup(page.content, 'html.parser')

    # this is the id for the table on the variable codebook bage
    tbl = soup.find("table",{"id":"GridView1"})

    # get the table and load into a df
    df_var_mapping = pd.read_html(str(tbl))[0]
    return df_var_mapping


questionnaire_var_desc = get_variable_description(
    url = 'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Questionnaire&Cycle=2017-2020'
)

demo_var_desc = get_variable_description(
    url = 'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Demographics&Cycle=2017-2020'
)

exam_var_desc = get_variable_description(
    url = 'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Examination&Cycle=2017-2020'
)

df_var_mapping = pd.concat([questionnaire_var_desc,demo_var_desc,exam_var_desc])
df_var_mapping

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints
0,DID040,How old {was SP/were you} when a doctor or oth...,P_DIQ,Diabetes,2017,2020,Questionnaire,None
1,DID060,For how long {have you/has SP} been taking ins...,P_DIQ,Diabetes,2017,2020,Questionnaire,None
2,DID250,How many times {have you/has SP} seen this doc...,P_DIQ,Diabetes,2017,2020,Questionnaire,None
3,DID260,How often {do you check your/does SP check his...,P_DIQ,Diabetes,2017,2020,Questionnaire,None
4,DID310D,What does {your/SP's} doctor or other health p...,P_DIQ,Diabetes,2017,2020,Questionnaire,None
...,...,...,...,...,...,...,...,...
1539,LUXCPIQR,Controlled attenuated parameter (CAP) interqua...,P_LUX,Liver Ultrasound Transient Elastography,2017,2020,Examination,None
1540,LUXSIQR,Stiffness (E) interquartile range (IQRe) of fi...,P_LUX,Liver Ultrasound Transient Elastography,2017,2020,Examination,None
1541,LUXSIQRM,Ratio of the stiffness IQRe / median E stiffne...,P_LUX,Liver Ultrasound Transient Elastography,2017,2020,Examination,None
1542,LUXSMED,Median liver stiffness (E). This indicator is ...,P_LUX,Liver Ultrasound Transient Elastography,2017,2020,Examination,None


In [29]:
df_var_mapping.to_csv("df_var_mapping.csv")

In [30]:
# create a key value mapping between the variable name and description
var_map = dict(zip(
    df_var_mapping['Variable Name'], 
    df_var_mapping['Variable Description']
))

var_map

{'DID040': 'How old {was SP/were you} when a doctor or other health professional first told {you/him/her} that {you/he/she} had diabetes or sugar diabetes?',
 'DID060': 'For how long {have you/has SP} been taking insulin?',
 'DID250': 'How many times {have you/has SP} seen this doctor or other health professional in the past 12 months?',
 'DID260': 'How often {do you check your/does SP check his/her} blood for glucose or sugar? Include times when checked by a family member or friend, but do not include times when checked by a doctor or other health professional.',
 'DID310D': "What does {your/SP's} doctor or other health professional say {your/his/her} blood pressure should be?",
 'DID310S': "What does {your/SP's} doctor or other health professional say {your/his/her} blood pressure should be?",
 'DID320': "One part of total serum cholesterol in {your/SP's} blood is a bad cholesterol, called LDL, which builds up and clogs {your/his/her} arteries. What was {your/his/her} most recent LDL

In [37]:

def read_xpt_files(folder_path):
    """Input folder path to read multiple xpt files in folder
       Returns dictionary with key: file name, 
                               value: df"""
    df_dict = {}
    for file_name in os.listdir(folder_path):
        if 'XPT' in os.path.splitext(file_name)[1]:
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_sas(file_path, format='xport')
            name = os.path.splitext(file_name)[0]
            if df.empty:
                raise Exception (f'Empty dataframe from file: {name}')
            df_dict[name] = df
        else:
            print(f'not loading file {file_name}')
    return df_dict



def full_outer_join(dataframes):
    joined_df = None
    for df in dataframes.values():
        if joined_df is None:
            joined_df = df
        else:
            joined_df = pd.merge(joined_df, df, on='SEQN', how='outer')
    return joined_df


folder_path = './xpt_data/'
dataframes_dict = read_xpt_files(folder_path)
df_cdc_joined = full_outer_join(dataframes_dict)
df_cdc_joined

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,ALQ170,...,SMD057,SMQ078,SMD641,SMD650,SMD100FL,SMD100MN,SMQ670,SMQ621,SMD630,SMAQUEX2
0,109266.0,1.0,1.000000e+01,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0
2,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0
3,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0
4,109273.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,1.0,30.0,15.0,1.0,1.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32957,121170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32958,121709.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32959,122778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32960,122879.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
dataframes_dict.keys()

dict_keys(['P_ALQ', 'P_RHQ', 'P_BMX', 'P_PAQ', 'P_DPQ', 'P_OCQ', 'P_WHQ', 'P_WHQMEC', 'P_DIQ', 'P_RXQ_RX', 'P_DEMO', 'P_SMQRTU', 'P_DBQ', 'P_HIQ', 'P_BPQ', 'P_PAQY', 'P_FSQ', 'P_HUQ', 'P_CDQ', 'P_INQ', 'P_MCQ', 'P_SMQ'])

# QA data

In [79]:
XPT_files = pd.DataFrame(columns = ['file_name', 'count_seqn', 'count_seqn_duplicate'])
SEQN_list = []


for key in dataframes_dict.keys():
    file_name = key
    count_seqn = len(dataframes_dict.get(key)['SEQN'].unique())
    count_seqn_duplicate = sum(dataframes_dict.get(key)['SEQN'].duplicated())
    XPT_file = pd.DataFrame([[file_name,count_seqn,count_seqn_duplicate]], columns=['file_name', 'count_seqn', 'count_seqn_duplicate'])
    XPT_files = XPT_files.append(XPT_file)
    SEQN_list.extend(dataframes_dict.get(key)['SEQN'].unique())
    

In [88]:
print(f"There is a total of {len(dataframes_dict.keys())} files")

There is a total of 22 files


In [89]:
print(f"There is a total of {len(np.unique(SEQN_list))} respondents")

There is a total of 15560 respondents


In [86]:
XPT_files

,file_name,count_seqn,count_seqn_duplicate
0,P_ALQ,8965,0
0,P_RHQ,5314,0
0,P_BMX,14300,0
0,P_PAQ,9693,0
0,P_DPQ,8965,0
0,P_OCQ,10195,0
0,P_WHQ,10195,0
0,P_WHQMEC,2211,0
0,P_DIQ,14986,0
0,P_RXQ_RX,15560,17402


# convert bytes and clean columns

In [38]:
# convert bytes and strip whitespace
df_cdc_joined_clean = df_cdc_joined.apply(lambda x: x.str.decode('utf-8').str.strip() if x.dtype == "object" else x)
# replace empty strings with nan
df_cdc_joined_clean = df_cdc_joined_clean.replace('', np.nan)
df_cdc_joined_clean

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,ALQ170,...,SMD057,SMQ078,SMD641,SMD650,SMD100FL,SMD100MN,SMQ670,SMQ621,SMD630,SMAQUEX2
0,109266.0,1.0,1.000000e+01,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0
2,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0
3,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0
4,109273.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,1.0,30.0,15.0,1.0,1.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32957,121170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32958,121709.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32959,122778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32960,122879.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# to use in R
df_cdc_joined_clean.to_csv("df_cdc_joined_clean.csv")

# filter to columns we care about

In [7]:
cols_to_keep = '''SEQN
FSD652CW
HUQ010
HUQ030
HUQ090
DPQ010
DPQ020
DPQ030
DPQ040
DPQ050
DPQ060
DPQ070
DPQ080
DPQ090
DPQ100
RXDUSE
RXDDAYS
RXDRSC1
RXDRSC2
RXDRSC3
RXDRSD1
RXDRSD2
RXDRSD3
RHQ074
RHQ076
RHD167
RHQ171'''.split()
cols_to_keep

['SEQN',
 'FSD652CW',
 'HUQ010',
 'HUQ030',
 'HUQ090',
 'DPQ010',
 'DPQ020',
 'DPQ030',
 'DPQ040',
 'DPQ050',
 'DPQ060',
 'DPQ070',
 'DPQ080',
 'DPQ090',
 'DPQ100',
 'RXDUSE',
 'RXDDAYS',
 'RXDRSC1',
 'RXDRSC2',
 'RXDRSC3',
 'RXDRSD1',
 'RXDRSD2',
 'RXDRSD3',
 'RHQ074',
 'RHQ076',
 'RHD167',
 'RHQ171']

# rename based on the mapping obtained from cdc codebook

In [8]:
df_cdc_joined_clean_trim = df_cdc_joined_clean[cols_to_keep]
df_cdc_joined_clean_trim

,SEQN,FSD652CW,HUQ010,HUQ030,HUQ090,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,...,RXDRSC1,RXDRSC2,RXDRSC3,RXDRSD1,RXDRSD2,RXDRSD3,RHQ074,RHQ076,RHD167,RHQ171
0,109264.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109266.0,NaN,3.0,1.0,2.0,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
2,109277.0,NaN,3.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109279.0,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,F90,R41,NaN,Attention-deficit hyperactivity disorders,Other symptoms and signs involving cognitive f...,NaN,NaN,NaN,NaN,NaN
4,109279.0,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,G47.9,NaN,NaN,"Sleep disorder, unspecified",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32957,124806.0,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,L20.9,NaN,NaN,"Atopic dermatitis, unspecified",NaN,NaN,NaN,NaN,NaN,NaN
32958,124808.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,Z79.3,NaN,NaN,Long term (current) use of hormonal contracept...,NaN,NaN,NaN,NaN,NaN,NaN
32959,124816.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32960,124819.0,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# rename columns with var descriptions

In [34]:
df_cdc_joined_clean_trim.rename(columns=var_map, inplace=True)
df_cdc_joined_clean_trim

,Respondent sequence number.,"After your {last} child was born, did you use WIC benefits to buy food for yourself?",{First/Next} I have some general questions about {your/SP's} health. Would you say {your/SP's} health in general is . . .,Is there a place that {you/SP} usually {go/goes} when {you are/he/she is} sick or {you/s/he} need{s} advice about {your/his/her} health?,"During the past 12 months, that is since {DISPLAY CURRENT MONTH} of {DISPLAY LAST YEAR}, {have you/has SP} seen or talked to a mental health professional such as a psychologist, psychiatrist, psychiatric nurse or clinical social worker about {your/his/her} health?","Over the last 2 weeks, how often have you been bothered by the following problems: little interest or pleasure in doing things? Would you say...","[Over the last 2 weeks, how often have you been bothered by the following problems:] feeling down, depressed, or hopeless?","[Over the last 2 weeks, how often have you been bothered by the following problems:] trouble falling or staying asleep, or sleeping too much?","[Over the last 2 weeks, how often have you been bothered by the following problems:] feeling tired or having little energy?","[Over the last 2 weeks, how often have you been bothered by the following problems:] poor appetite or overeating?",...,ICD-10-CM code 1.,ICD-10-CM code 2.,ICD-10-CM code 3.,ICD-10-CM code 1 description.,ICD-10-CM code 2 description.,ICD-10-CM code 3 description.,The next questions are about {your/SP's} pregnancy history. {Have you/Has SP} ever attempted to become pregnant over a period of at least a year without becoming pregnant?,{Have you/Has SP} ever been to a doctor or other medical provider because {you have/she has} been unable to become pregnant?,How many deliveries {have you/has SP} had? (Please count all vaginal and Cesarean deliveries and count stillbirths as well as live births.),How many of {your/her} deliveries resulted {Did {your/her} delivery result} in a live birth?
0,109264.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109266.0,NaN,3.0,1.0,2.0,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
2,109277.0,NaN,3.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109279.0,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,F90,R41,NaN,Attention-deficit hyperactivity disorders,Other symptoms and signs involving cognitive f...,NaN,NaN,NaN,NaN,NaN
4,109279.0,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,G47.9,NaN,NaN,"Sleep disorder, unspecified",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32957,124806.0,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,L20.9,NaN,NaN,"Atopic dermatitis, unspecified",NaN,NaN,NaN,NaN,NaN,NaN
32958,124808.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,Z79.3,NaN,NaN,Long term (current) use of hormonal contracept...,NaN,NaN,NaN,NaN,NaN,NaN
32959,124816.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32960,124819.0,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:

with pd.option_context('display.max_columns', None, 'display.float_format', lambda x: '%.3f' % x): # also can do all rows using 'display.max_rows', None, 
    display(df_cdc_joined_clean_trim.describe())

,Respondent sequence number.,"After your {last} child was born, did you use WIC benefits to buy food for yourself?",{First/Next} I have some general questions about {your/SP's} health. Would you say {your/SP's} health in general is . . .,Is there a place that {you/SP} usually {go/goes} when {you are/he/she is} sick or {you/s/he} need{s} advice about {your/his/her} health?,"During the past 12 months, that is since {DISPLAY CURRENT MONTH} of {DISPLAY LAST YEAR}, {have you/has SP} seen or talked to a mental health professional such as a psychologist, psychiatrist, psychiatric nurse or clinical social worker about {your/his/her} health?","Over the last 2 weeks, how often have you been bothered by the following problems: little interest or pleasure in doing things? Would you say...","[Over the last 2 weeks, how often have you been bothered by the following problems:] feeling down, depressed, or hopeless?","[Over the last 2 weeks, how often have you been bothered by the following problems:] trouble falling or staying asleep, or sleeping too much?","[Over the last 2 weeks, how often have you been bothered by the following problems:] feeling tired or having little energy?","[Over the last 2 weeks, how often have you been bothered by the following problems:] poor appetite or overeating?","[Over the last 2 weeks, how often have you been bothered by the following problems:] feeling bad about yourself - or that you are a failure or have let yourself or your family down?","[Over the last 2 weeks, how often have you been bothered by the following problems:] trouble concentrating on things, such as reading the newspaper or watching TV?","[Over the last 2 weeks, how often have you been bothered by the following problems:] moving or speaking so slowly that other people could have noticed? Or the opposite - being so fidgety or restless that you have been moving around a lot more than usual?","[Over the last 2 weeks, how often have you been bothered by the following problems:] Thoughts that you would be better off dead or of hurting yourself in some way?","How difficult have these problems made it for you to do your work, take care of things at home, or get along with people?","In the past 30 days, have you used or taken medication for which a prescription is needed? Do not include prescription vitamins or minerals you may have already told me about.",For how long have you been using or taking {PRODUCT NAME}?,The next questions are about {your/SP's} pregnancy history. {Have you/Has SP} ever attempted to become pregnant over a period of at least a year without becoming pregnant?,{Have you/Has SP} ever been to a doctor or other medical provider because {you have/she has} been unable to become pregnant?,How many deliveries {have you/has SP} had? (Please count all vaginal and Cesarean deliveries and count stillbirths as well as live births.),How many of {your/her} deliveries resulted {Did {your/her} delivery result} in a live birth?
count,32962.000,379.000,32962.000,32962.000,31113.000,22468.000,22450.000,22450.000,22447.000,22447.000,22446.000,22446.000,22446.000,22433.000,16304.000,32962.000,24031.000,5600.000,5600.000,10231.000,9777.000
mean,117047.003,1.865,2.891,1.092,1.848,0.527,0.508,0.841,0.959,0.527,0.345,0.378,0.256,0.089,0.446,1.267,4175.267,1.899,1.926,2.697,2.682
std,4489.493,1.313,1.151,0.335,0.387,0.925,0.907,1.119,1.073,0.934,0.829,0.838,0.758,0.482,0.783,0.465,14172.014,0.399,0.387,2.829,1.811
min,109263.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000,1.000,0.000,0.000
25%,113128.000,1.000,2.000,1.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,365.000,2.000,2.000,2.000,2.000
50%,117020.500,2.000,3.000,1.000,2.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1095.000,2.000,2.000,2.000,2.000
75%,120928.750,2.000,4.000,1.000,2.000,1.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,1.000,2.000,3650.000,2.000,2.000,4.000,4.000
m